In [1]:
import os
from pathlib import Path
os.chdir('../')
%pwd

'c:\\Users\\Saumil\\Git\\text_summarizer'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path
    max_input_length: int
    max_output_length: int

In [3]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_dir

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_dir([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name,
            max_input_length = config.max_input_length,
            max_output_length = config.max_output_length
        )
        
        return data_transformation_config

In [14]:
import os
from transformers import AutoTokenizer
from textSummarizer.logging import logger
from datasets import load_dataset, load_from_disk

In [17]:

class DataTransfomation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, examples):
        input_encoding = self.tokenizer(examples['dialogue'], truncation=True, padding=True, max_length=self.config.max_input_length)

        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(examples['summary'], truncation=True, padding=True, max_length=self.config.max_output_length)

        return {
            'input_ids': input_encoding['input_ids'],
            'attention_mask': input_encoding['attention_mask'],
            'labels': target_encoding['input_ids']        
        }
    
    def convert(self):
        data_samsum = load_from_disk(self.config.data_path)
        data_samsum_pt = data_samsum.map(self.convert_examples_to_features, batched=True)
        data_samsum_pt.save_to_disk(Path(f'{self.config.root_dir}/samsum_dataset'))

In [24]:
try:
    config = ConfigurationManager()
    data_transformation = DataTransfomation(config.get_data_transformation_config())
    data_transformation.convert()
    '''logger.info('configuration is created')'''
except Exception as e:
    raise e

[2024-01-21 16:04:55,037: INFO: common: Read config\config.yaml successfully]
[2024-01-21 16:04:55,040: INFO: common: Read params.yaml successfully]
[2024-01-21 16:04:55,041: INFO: common: artifacts/data_transformation created]


Saving the dataset (1/1 shards): 100%|██████████| 819/819 [00:00<00:00, 341362.91 examples/s]


In [26]:
load_dataset('artifacts/data_ingestion/samsum_dataset')

DatasetDict({
    train: Dataset({
        features: ['dialogue', 'summary', 'id'],
        num_rows: 14732
    })
    validation: Dataset({
        features: ['dialogue', 'summary', 'id'],
        num_rows: 818
    })
    test: Dataset({
        features: ['dialogue', 'summary', 'id'],
        num_rows: 819
    })
})